# 가설검정

## 귀무가설과 대립가설
- 귀무가설(H0): 효과/차이 없음
- 대립가설(H1): 효과/차이 있음

## 검정 결과 해석
- 검정통계량: 데이터가 H0와 얼마나 다른지 나타내는 값
- p-value (예: α=0.05)
  - p < α  -> H0 기각 (H1 채택)
  - p ≥ α -> H0 기각 불가 (H0 채택)

## 가설 검정 프로세스
1) 가설 설정: H0, H1
2) 유의수준 결정: α=0.05 등
3) 검정통계량 및 p-value 계산
4) 결과 도출 및 해석

# t-검정(t-test) 개요

## 유형 요약
| 검정 유형            | 표본/모집단 구조        | 비교 관계        | 예시                         |
|---------------------|-------------------------|------------------|------------------------------|
| 단일 표본 t-검정     | 모집단 1, 표본 1        | 한 그룹 vs 기준값 | 과자 무게가 200g과 다른지    |
| 대응(쌍체) t-검정    | 같은 집단 전후/짝 자료  | 같은 그룹(쌍)     | 신약 전후 효과               |
| 독립 표본 t-검정     | 서로 다른 두 집단       | 서로 다른 그룹    | 1반 vs 2반 성적 차이         |

## 단일 표본 t-검정 (One-sample)
- 정규성 검정: shapiro(data)
  - Yes -> ttest_1samp(data, popmean)
  - No  -> wilcoxon(data - popmean)

## 대응(쌍체) t-검정 (Paired)
- diff = data1 - data2, 정규성: shapiro(diff)
  - Yes -> ttest_rel(data1, data2)
  - No  -> wilcoxon(data1, data2)

## 독립 표본 t-검정 (Independent)
- 정규성: shapiro(data1), shapiro(data2)
  - Yes -> 등분산: levene(data1, data2)
      - 등분산 Yes -> ttest_ind(data1, data2)
      - 등분산 No  -> ttest_ind(data1, data2, equal_var=False)   # Welch
  - No  -> mannwhitneyu(data1, data2)

## SciPy 함수 요약
- shapiro(x), levene(x,y)
- ttest_1samp(x, popmean)
- ttest_rel(x, y)
- ttest_ind(x, y, equal_var=True|False)
- wilcoxon(x, y) 또는 wilcoxon(x - y)
- mannwhitneyu(x, y)
